In [3]:
import pandas as pd
import numpy as np


In [1]:
a = {
    "buildship": "https://github.com/eclipse/steady.git",
    "eclips-collections": "https://github.com/eclipse/eclipse-collections.git",
    "hawkbit": "https://github.com/eclipse/hawkbit.git",
    "jkube": "https://github.com/eclipse/jkube.git",
    "kura": "https://github.com/eclipse/kura.git",
    "jifa": "https://github.com/eclipse/microprofile.git",
    "milo": "https://github.com/eclipse/milo.git",
    "openvsx": "https://github.com/eclipse/openvsx.git",
    "steady": "https://github.com/eclipse/steady.git",
    "xtext": "https://github.com/eclipse/xtext.git"
}

In [7]:
# Step 3: Adjust the 'category' field based on conditions
def adjust_category(row):
    if pd.notna(row['category_df1']) and row['category_df1'] == "DOS":
        return "DOS" if pd.isna(row['category_df2']) else f"DOS,{row['category_df2']}"
    return row['category_df2'] if pd.notna(row['category_df2']) else ""


def merge_dataframes(df1, df2):
    # Step 1: Rename 'label' in df1 to 'category' for uniformity and drop 'label' from df1 if it exists
    df1 = df1.rename(columns={'label': 'category'})
    df1['category'] = df1['category'].apply(lambda x: "DOS" if x == "positive" else "")

    # Step 2: Merge the DataFrames on specified columns
    merged_df = pd.merge(df1, df2, 
                        on=["content", "method_name", "parameters", "access_modifier"], 
                        how='outer', 
                        suffixes=('_df1', '_df2'))



    

MODEl Implementation (Multi label model)

In [2]:
import ktrain
import pandas as pd
from ktrain import tabular
import numpy as np
import pandas   as pd
from sklearn.utils import shuffle
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [3]:
def get_model(a,project):
    # load the model
    df = pd.read_csv(f'../data/{project}_final.csv')
    # drop all rows in df
    df = df.drop(df.index)
    print(df.shape)
    for p,u in a.items():
        if p == project:
            continue
        df1 = pd.read_csv(f'../data/{p}_final.csv')
        df = pd.concat([df,df1])
    df = df.drop(columns=['comments'],axis=1)

    
    train_df = shuffle(df)
    test_df = pd.read_csv(f'../data/{project}_final.csv')
    test_df = test_df.drop(columns=['comments'],axis=1)

    test_df['index'] = test_df.index
    train_df['index'] = train_df.index  
    return train_df, test_df

In [10]:
def get_lean(train_df,test_df):
    print(test_df.iloc[1])
    trn,val,preproc = tabular.tabular_from_df(train_df, label_columns=['is','set','get','DOS'], random_state=42)
    model  = tabular.tabular_classifier(name='mlp',multilabel=True,train_data=trn,hidden_layers=[1000,1000,500],hidden_dropouts=[0.2,0.5,0.5])
    learner = ktrain.get_learner(model,train_data=trn,val_data=val,batch_size=32)
    # learner.lr_find(show_plot=True, max_epochs=5)
    # data = {"get":0,"is":1,"not_LA":2,"set":3}
    print(preproc.get_classes())
    learner.fit_onecycle(5e-3, 1)
    print(learner.evaluate(val, class_names=preproc.get_classes()))
    predictor = ktrain.get_predictor(learner.model, preproc)
    return predictor

In [5]:
def model_save(predictor,test_df,output_file):
    print()
    print()
    model_path = f'model/{output_file}_final_model'
    print(model_path)
    print()
    print()
    predictor.save(model_path)

In [11]:
a = {
    "buildship": " ", 
    "eclips-collections": " ",
    "jifa": " ",
    "jkube": " ",
    "hawkbit": "https://github.com/eclipse/hawkbit.git",
    "kura": "https://github.com/eclipse/kura.git",
    "milo": "https://github.com/eclipse/milo.git",
    "openvsx": "https://github.com/eclipse/openvsx.git",
    "steady": "https://github.com/eclipse/steady.git",
    "xtext": "https://github.com/eclipse/xtext.git"
}
for project,url in a.items():
    print(f"the project is {project}")
    train_df,test_df = get_model(a,project)
    predictor = get_lean(train_df,test_df)
    model_save(predictor,test_df,project)
    break

the project is buildship
(0, 11)
access_modifier                                               static
method_name                                           getArtifactUri
parameters                              String g, String a, String v
content            private static String getArtifactUri(String g,...
return_type                                                     void
returns                                                   return s; 
is                                                                 0
set                                                                0
get                                                                1
DOS                                                                0
index                                                              1
Name: 1, dtype: object
processing train: 130271 rows x 11 columns
['is', 'set', 'get', 'DOS']
       is  set  get  DOS
2257    0    0    0    1
16950   0    0    0    0
70485   0    0    0    0
87935   0    0

/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/ktrain/core.py:226: UserWarning: Confusion matrices do not currently support multilabel classification, so returning None
  warnings.warn(
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving yo

In [2]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 7 commits.
  (use "git push" to publish your local commits)

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   DecisionTree_content.ipynb
	modified:   Logistic_regression_content.ipynb
	modified:   SVM_content.ipynb
	new file:   adaboost.ipynb
	modified:   app.py
	modified:   code_bert copy.ipynb
	modified:   code_bert_execution_part1.ipynb
	modified:   eval.ipynb
	modified:   gpt_labeling.ipynb
	modified:   knn_content.ipynb
	new file:   kura
	new file:   kura_info.txt
	modified:   merge.ipynb
	modified:   merge_content.ipynb
	modified:   merge_features.ipynb
	modified:   milo_eval.csv
	new file:   model/buildship_final_model/tf_model.h5
	new file:   model/buildship_final_model/tf_model.preproc
	new file:   suject_application_states.png
	modified:   temp1_featers.ipynb

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>.

In [4]:
!git add .

hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> code/ds
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached code/ds
hint: 
hint: See "git help submodule" for more information.


In [6]:
!git commit -m "completed code"

[main f48bc58] completed code
 33 files changed, 324330 insertions(+), 374283 deletions(-)
 create mode 100644 code/adaboost.ipynb
 create mode 100644 code/cda_info.txt
 create mode 100644 code/da_info.txt
 create mode 160000 code/ds
 create mode 100644 code/ds_info.txt
 create mode 160000 code/kura
 create mode 100644 code/kura_info.txt
 create mode 160000 code/milo
 delete mode 100644 code/milo1_data.csv
 delete mode 100644 code/milo2_data.csv
 delete mode 100644 code/milo3_data.csv
 delete mode 100644 code/milo4_data.csv
 delete mode 100644 code/milo5_data.csv
 create mode 100644 code/model/buildship_final_model/tf_model.h5
 create mode 100644 code/model/buildship_final_model/tf_model.preproc
 create mode 100644 code/sa_info.txt
 create mode 100644 code/suject_application_states.png


In [7]:
!git push

Enumerating objects: 229, done.
Counting objects: 100% (229/229), done.
Delta compression using up to 8 threads
Compressing objects: 100% (192/192), done.
error: RPC failed; HTTP 400 curl 22 The requested URL returned error: 400
send-pack: unexpected disconnect while reading sideband packet
Writing objects: 100% (213/213), 12.43 GiB | 47.87 MiB/s, done.
Total 213 (delta 105), reused 19 (delta 0), pack-reused 0
fatal: the remote end hung up unexpectedly
Everything up-to-date
